In [30]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
%cd /content/drive/MyDrive/Neo4j

/content/drive/MyDrive/Neo4j


In [32]:
!pwd

/content/drive/MyDrive/Neo4j


In [33]:
!pip install langchain langchain-community langchain-core langchain-experimental langchain-openai langchainhub langsmith py2neo neo4j openai jupyter notebook pandas

In [34]:
import os
import pandas as pd

from langchain_community.graphs import Neo4jGraph

# In local pc - to access env variables
# from dotenv import load_dotenv
# _ = load_dotenv()
from google.colab import userdata # For colab env variables

In [35]:
os.environ["NEO4J_USERNAME"] = userdata.get('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')
os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')

In [36]:
df = pd.read_csv('vgsales.csv')

In [37]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [38]:
df['Rank'].nunique()==df.shape[0]

True

In [39]:
kg = Neo4jGraph()

In [40]:
vgsales_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/KNAI-AI/knai-workshop/main/data/vgsales.csv' AS row
MERGE (g:Game {rank:toInteger(row.Rank)})
SET g.name = row.Name,
    g.platform = row.Platform,
    g.year = toInteger(row.Year),
    g.genre = row.Genre,
    g.publisher = row.Publisher,
    g.na_sales = toFloat(row.NA_Sales),
    g.eu_sales = toFloat(row.EU_Sales),
    g.jp_sales = toFloat(row.JP_Sales),
    g.other_sales = toFloat(row.Other_Sales),
    g.global_sales = toFloat(row.Global_Sales)

MERGE (p:Platform {name:row.Platform})
MERGE (g)-[:RELEASED_ON]->(p)

MERGE (pub:Publisher {name:row.Publisher})
MERGE (g)-[:PUBLISHED_BY]->(pub)

MERGE (ge:Genre {name:row.Genre})
MERGE (g)-[:BELONGS_TO_GENRE]->(ge)
"""
kg.query(vgsales_query)

[]

In [ ]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Game {rank: INTEGER, name: STRING, platform: STRING, year: INTEGER, genre: STRING, publisher: STRING, na_sales: FLOAT, eu_sales: FLOAT, jp_sales: FLOAT, other_sales: FLOAT, global_sales: FLOAT}
Platform {name: STRING}
Publisher {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Game)-[:RELEASED_ON]->(:Platform)
(:Game)-[:PUBLISHED_BY]->(:Publisher)
(:Game)-[:BELONGS_TO_GENRE]->(:Genre)


In [ ]:
cypher = """
  MATCH (n)
  RETURN count(n) AS numberOfNodes
  """
result = kg.query(cypher)
result

[{'numberOfNodes': 17220}]

In [ ]:
df['Rank'].nunique()+df['Platform'].nunique()+df['Genre'].nunique() + df['Publisher'].nunique()

17219

In [ ]:
print(f"There are {result[0]['numberOfNodes']} nodes in this graph.")

There are 17220 nodes in this graph.


In [ ]:
cypher = """
  MATCH (n:Game)
  RETURN count(n) AS numberOfGames
  """
kg.query(cypher)

[{'numberOfGames': 16598}]

In [ ]:
cypher = """
  MATCH (nintendo:Publisher {name:"Nintendo"})
  RETURN count(nintendo) as count_nintendo
  """
kg.query(cypher)

[{'count_nintendo': 1}]

In [ ]:
cypher = """
  MATCH (gta:Game {name:"Grand Theft Auto: San Andreas"})
  RETURN gta.global_sales
  """
kg.query(cypher)

[{'gta.global_sales': 20.81},
 {'gta.global_sales': 1.95},
 {'gta.global_sales': 0.98},
 {'gta.global_sales': 0.12}]

In [ ]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
df[df['Name']=='Grand Theft Auto: San Andreas']

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
17,18,Grand Theft Auto: San Andreas,PS2,2004.0,Action,Take-Two Interactive,9.43,0.40,0.41,10.57,20.81
873,875,Grand Theft Auto: San Andreas,XB,2005.0,Action,Take-Two Interactive,1.26,0.61,0.00,0.09,1.95
2120,2122,Grand Theft Auto: San Andreas,PC,2005.0,Action,Take-Two Interactive,0.00,0.92,0.00,0.05,0.98
9827,9829,Grand Theft Auto: San Andreas,X360,2008.0,Action,Take-Two Interactive,0.08,0.03,0.00,0.01,0.12


In [ ]:
cypher = """
  MATCH (marioBros:Game {name:"New Super Mario Bros."})
  RETURN marioBros.global_sales, marioBros.publisher
  """
kg.query(cypher)

[{'marioBros.global_sales': 30.01, 'marioBros.publisher': 'Nintendo'}]

In [ ]:
cypher = """
    MATCH (g:Game)
    RETURN g.platform AS platform, SUM(g.global_sales) AS totalGlobalSales
    ORDER BY totalGlobalSales DESC
  """

kg.query(cypher)

[{'platform': 'PS2', 'totalGlobalSales': 1255.6399999999871},
 {'platform': 'X360', 'totalGlobalSales': 979.9599999999996},
 {'platform': 'PS3', 'totalGlobalSales': 957.8399999999987},
 {'platform': 'Wii', 'totalGlobalSales': 926.7099999999971},
 {'platform': 'DS', 'totalGlobalSales': 822.4899999999874},
 {'platform': 'PS', 'totalGlobalSales': 730.659999999997},
 {'platform': 'GBA', 'totalGlobalSales': 318.499999999998},
 {'platform': 'PSP', 'totalGlobalSales': 296.2799999999948},
 {'platform': 'PS4', 'totalGlobalSales': 278.0999999999994},
 {'platform': 'PC', 'totalGlobalSales': 258.81999999999846},
 {'platform': 'XB', 'totalGlobalSales': 258.25999999999834},
 {'platform': 'GB', 'totalGlobalSales': 255.44999999999987},
 {'platform': 'NES', 'totalGlobalSales': 251.06999999999988},
 {'platform': '3DS', 'totalGlobalSales': 247.4600000000001},
 {'platform': 'N64', 'totalGlobalSales': 218.87999999999985},
 {'platform': 'SNES', 'totalGlobalSales': 200.05000000000024},
 {'platform': 'GC', 't

In [ ]:
cypher = """
  MATCH (nineties:Game)
  WHERE nineties.year >= 1990
    AND nineties.year < 2000
  RETURN nineties.name
  """
kg.query(cypher)

[{'nineties.name': 'Pokemon Red/Pokemon Blue'},
 {'nineties.name': 'Pokemon Gold/Pokemon Silver'},
 {'nineties.name': 'Super Mario World'},
 {'nineties.name': 'Pokémon Yellow: Special Pikachu Edition'},
 {'nineties.name': 'Super Mario 64'},
 {'nineties.name': 'Super Mario Land 2: 6 Golden Coins'},
 {'nineties.name': 'Gran Turismo'},
 {'nineties.name': 'Super Mario All-Stars'},
 {'nineties.name': 'Mario Kart 64'},
 {'nineties.name': 'Final Fantasy VII'},
 {'nineties.name': 'Gran Turismo 2'},
 {'nineties.name': 'Donkey Kong Country'},
 {'nineties.name': 'Super Mario Kart'},
 {'nineties.name': 'GoldenEye 007'},
 {'nineties.name': 'Final Fantasy VIII'},
 {'nineties.name': 'The Legend of Zelda: Ocarina of Time'},
 {'nineties.name': 'Crash Bandicoot 2: Cortex Strikes Back'},
 {'nineties.name': 'Tekken 3'},
 {'nineties.name': 'Crash Bandicoot 3: Warped'},
 {'nineties.name': 'Crash Bandicoot'},
 {'nineties.name': 'Street Fighter II: The World Warrior'},
 {'nineties.name': 'Driver'},
 {'ninetie

In [ ]:
cypher = """
  MATCH (game:Game)-[:BELONGS_TO_GENRE]->(genre:Genre)
  RETURN game.name, genre.name limit 10
  """
kg.query(cypher)

[{'game.name': 'Wii Sports', 'genre.name': 'Sports'},
 {'game.name': 'Wii Sports Resort', 'genre.name': 'Sports'},
 {'game.name': 'Wii Fit', 'genre.name': 'Sports'},
 {'game.name': 'Wii Fit Plus', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 16', 'genre.name': 'Sports'},
 {'game.name': 'Mario & Sonic at the Olympic Games', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 14', 'genre.name': 'Sports'},
 {'game.name': 'Zumba Fitness', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 12', 'genre.name': 'Sports'},
 {'game.name': 'FIFA 15', 'genre.name': 'Sports'}]

In [ ]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
cypher = """
  MATCH (game:Game {name:"New Super Mario Bros."})-[:PUBLISHED_BY]->(m)<-[:PUBLISHED_BY]-(otherGames)
  RETURN otherGames.name, otherGames.global_sales limit 10
  """
kg.query(cypher)

[{'otherGames.name': 'Wii Sports', 'otherGames.global_sales': 82.74},
 {'otherGames.name': 'Super Mario Bros.', 'otherGames.global_sales': 40.24},
 {'otherGames.name': 'Mario Kart Wii', 'otherGames.global_sales': 35.82},
 {'otherGames.name': 'Wii Sports Resort', 'otherGames.global_sales': 33.0},
 {'otherGames.name': 'Pokemon Red/Pokemon Blue',
  'otherGames.global_sales': 31.37},
 {'otherGames.name': 'Tetris', 'otherGames.global_sales': 30.26},
 {'otherGames.name': 'Wii Play', 'otherGames.global_sales': 29.02},
 {'otherGames.name': 'New Super Mario Bros. Wii',
  'otherGames.global_sales': 28.62},
 {'otherGames.name': 'Duck Hunt', 'otherGames.global_sales': 28.31},
 {'otherGames.name': 'Nintendogs', 'otherGames.global_sales': 24.76}]

In [ ]:
cypher = """
  MATCH (game:Game {name:"New Super Mario Bros."})-[:PUBLISHED_BY]->(m)<-[:PUBLISHED_BY]-(othergame)
  RETURN othergame.name, othergame.global_sales
  """
kg.query(cypher)

[]

In [ ]:
cypher = """
  MATCH (game:Game {name:"New Super Mario Bros."})-[:PUBLISHED_BY]->(m)<-[:PUBLISHED_BY]-(otherGames)
  return count(*) as rowcount
  """
kg.query(cypher)

[{'rowcount': 702}]

In [ ]:
# df[df['Name']=='New Super Mario Bros.']['Publisher'].values[0]
# df[df['Publisher']==df[df['Name']=='New Super Mario Bros.']['Publisher']]
a=df[df['Publisher']==df[df['Name']=='New Super Mario Bros.']['Publisher'].values[0]]


In [ ]:
a[a['Name']=='New Super Mario Bros.']

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
6,7,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.5,2.9,30.01


In [ ]:
a.shape[0]

703

In [43]:
import os

from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph


In [41]:
# In Google Colab
os.environ["NEO4J_USERNAME"] = userdata.get('NEO4J_USERNAME')
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')
os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [44]:
graph = Neo4jGraph()
cypher_llm = ChatOpenAI(model="gpt-4-turbo")
chain = GraphCypherQAChain.from_llm(graph=graph, llm=cypher_llm, verbose=True, validate_cypher=True)

In [45]:
response = chain.invoke({"query": "Which games have same publisher in the year 2003?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2003 AND g2.year = 2003 AND g1 <> g2
RETURN g1.name AS Game1, g2.name AS Game2, p.name AS Publisher

Full Context:
[{'Game1': 'Super Mario Bros. 3', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Pokémon Colosseum', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario & Luigi: Superstar Saga', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario Party 5', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Donkey Kong Country', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Kirby Air Ride', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Pokemon Pinball: Ruby & Sapphire', 'Game2': 'Mario Kart: Double Dash!!', 'Publisher': 'Nintendo'}, {'Game1': 'Mario Golf: Toadstool To

{'query': 'Which games have same publisher in the year 2003?',
 'result': 'Super Mario Bros. 3, Pokémon Colosseum, Mario & Luigi: Superstar Saga, Mario Party 5, Donkey Kong Country, Kirby Air Ride, Pokemon Pinball: Ruby & Sapphire, Mario Golf: Toadstool Tour, Final Fantasy: Crystal Chronicles, and WarioWare, Inc.: Mega MicroGame$ all share the same publisher, Nintendo, in the year 2003.'}

In [46]:
print(response['result'])

Super Mario Bros. 3, Pokémon Colosseum, Mario & Luigi: Superstar Saga, Mario Party 5, Donkey Kong Country, Kirby Air Ride, Pokemon Pinball: Ruby & Sapphire, Mario Golf: Toadstool Tour, Final Fantasy: Crystal Chronicles, and WarioWare, Inc.: Mega MicroGame$ all share the same publisher, Nintendo, in the year 2003.


In [50]:
response = chain.invoke({"query": "Find pairs of games published by the same publisher in the year 2003 and their total combined global sales."})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2003 AND g2.year = 2003 AND g1.name <> g2.name
RETURN p.name AS Publisher, g1.name AS Game1, g2.name AS Game2, (g1.global_sales + g2.global_sales) AS TotalGlobalSales

Full Context:
[{'Publisher': 'Nintendo', 'Game1': 'Super Mario Bros. 3', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 12.15}, {'Publisher': 'Nintendo', 'Game1': 'Pokémon Colosseum', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 9.49}, {'Publisher': 'Nintendo', 'Game1': 'Mario & Luigi: Superstar Saga', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 9.120000000000001}, {'Publisher': 'Nintendo', 'Game1': 'Mario Party 5', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 9.030000000000001}, {'Publisher': 'Nintendo', 'Game1': 'Donkey Kong Country', 'Game2': 'Mario Kart: Double Dash!!', 'TotalGlobalSales': 8.99}, {'Publisher':

{'query': 'Find pairs of games published by the same publisher in the year 2003 and their total combined global sales.',
 'result': 'The pairs of games published by Nintendo in 2003 along with their total combined global sales are as follows:\n- "Super Mario Bros. 3" and "Mario Kart: Double Dash!!" with total global sales of 12.15 million.\n- "Pokémon Colosseum" and "Mario Kart: Double Dash!!" with total global sales of 9.49 million.\n- "Mario & Luigi: Superstar Saga" and "Mario Kart: Double Dash!!" with total global sales of 9.12 million.\n- "Mario Party 5" and "Mario Kart: Double Dash!!" with total global sales of 9.03 million.\n- "Donkey Kong Country" and "Mario Kart: Double Dash!!" with total global sales of 8.99 million.\n- "Kirby Air Ride" and "Mario Kart: Double Dash!!" with total global sales of 8.57 million.\n- "Pokemon Pinball: Ruby & Sapphire" and "Mario Kart: Double Dash!!" with total global sales of 8.51 million.\n- "Mario Golf: Toadstool Tour" and "Mario Kart: Double Dash

In [51]:
print(response['result'])

The pairs of games published by Nintendo in 2003 along with their total combined global sales are as follows:
- "Super Mario Bros. 3" and "Mario Kart: Double Dash!!" with total global sales of 12.15 million.
- "Pokémon Colosseum" and "Mario Kart: Double Dash!!" with total global sales of 9.49 million.
- "Mario & Luigi: Superstar Saga" and "Mario Kart: Double Dash!!" with total global sales of 9.12 million.
- "Mario Party 5" and "Mario Kart: Double Dash!!" with total global sales of 9.03 million.
- "Donkey Kong Country" and "Mario Kart: Double Dash!!" with total global sales of 8.99 million.
- "Kirby Air Ride" and "Mario Kart: Double Dash!!" with total global sales of 8.57 million.
- "Pokemon Pinball: Ruby & Sapphire" and "Mario Kart: Double Dash!!" with total global sales of 8.51 million.
- "Mario Golf: Toadstool Tour" and "Mario Kart: Double Dash!!" with total global sales of 8.48 million.
- "Final Fantasy: Crystal Chronicles" and "Mario Kart: Double Dash!!" with total global sales of

In [52]:
response = chain.invoke({"query": "Identify publishers who released games in both 2001 and 2002, and compare their total global sales for each year"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g1:Game)-[:PUBLISHED_BY]->(p:Publisher)<-[:PUBLISHED_BY]-(g2:Game)
WHERE g1.year = 2001 AND g2.year = 2002
WITH p.name AS Publisher, 
     SUM(CASE WHEN g1.year = 2001 THEN g1.global_sales ELSE 0 END) AS Sales2001,
     SUM(CASE WHEN g2.year = 2002 THEN g2.global_sales ELSE 0 END) AS Sales2002
RETURN Publisher, Sales2001, Sales2002

Full Context:
[{'Publisher': 'Nintendo', 'Sales2001': 998.139999999999, 'Sales2002': 1062.820000000005}, {'Publisher': 'Microsoft Game Studios', 'Sales2001': 269.8000000000002, 'Sales2002': 69.59999999999995}, {'Publisher': 'Take-Two Interactive', 'Sales2001': 163.44000000000005, 'Sales2002': 243.48000000000022}, {'Publisher': 'Sony Computer Entertainment', 'Sales2001': 1298.700000000001, 'Sales2002': 982.300000000004}, {'Publisher': 'Activision', 'Sales2001': 929.4999999999981, 'Sales2002': 596.6199999999945}, {'Publisher': 'Ubisoft', 'Sales2001': 219.56000000000003, 'Sales2002': 

{'query': 'Identify publishers who released games in both 2001 and 2002, and compare their total global sales for each year',
 'result': "In 2001 and 2002, several publishers released games, showing varied sales figures across both years. Here's a comparison of their total global sales:\n\n- Nintendo had sales of approximately $998.14 million in 2001 and $1062.82 million in 2002.\n- Microsoft Game Studios recorded sales of about $269.80 million in 2001 and $69.60 million in 2002.\n- Take-Two Interactive posted sales of $163.44 million in 2001 and increased to $243.48 million in 2002.\n- Sony Computer Entertainment saw sales of $1298.70 million in 2001, which decreased to $982.30 million in 2002.\n- Activision reported sales of $929.50 million in 2001 and $596.62 million in 2002.\n- Ubisoft had $219.56 million in sales in 2001 and rose to $405.25 million in 2002.\n- Electronic Arts achieved sales of $3970.56 million in 2001 and $3139.43 million in 2002.\n- Sega's sales were $257.40 mill

In [53]:
print(response['result'])

In 2001 and 2002, several publishers released games, showing varied sales figures across both years. Here's a comparison of their total global sales:

- Nintendo had sales of approximately $998.14 million in 2001 and $1062.82 million in 2002.
- Microsoft Game Studios recorded sales of about $269.80 million in 2001 and $69.60 million in 2002.
- Take-Two Interactive posted sales of $163.44 million in 2001 and increased to $243.48 million in 2002.
- Sony Computer Entertainment saw sales of $1298.70 million in 2001, which decreased to $982.30 million in 2002.
- Activision reported sales of $929.50 million in 2001 and $596.62 million in 2002.
- Ubisoft had $219.56 million in sales in 2001 and rose to $405.25 million in 2002.
- Electronic Arts achieved sales of $3970.56 million in 2001 and $3139.43 million in 2002.
- Sega's sales were $257.40 million in 2001, dropping to $110.70 million in 2002.
- SquareSoft had minimal sales of $0.50 million in 2001 and $0.12 million in 2002.
- Atari report